# Clasificación con Red Neuronal

In [ ]:
import numpy as np
import tensorflow as tf

### Importar Dataset y prepararlo

In [ ]:
# Importar dataset y dividirlo
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape, x_test.shape)

# Cambiar el tamaño de dataset (aplanar en un vector cada imagen)
x_train = x_train.reshape((len(x_train), x_train.shape[1] * x_train.shape[2]))
x_test = x_test.reshape((len(x_test), x_test.shape[1] * x_test.shape[2]))
print(x_train.shape, x_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (10000, 28, 28)
(60000, 784) (10000, 784)


### Normalizar los Datos

In [ ]:
# Para normalizar debe conocerse el valor máximo
print(x_train.max(), x_train.min())

255 0


In [ ]:
x_train.astype('float')
x_test.astype('float')
x_train = x_train/255
x_test = x_test/255
print(x_train.max(), x_train.min())

1.0 0.0


### Crear conjunto de validación

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de entrenamiento para tener uno de validación
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size = 0.83)

In [ ]:
print( x_train.shape, x_val.shape, x_test.shape)
print( y_train.shape, y_val.shape, y_test.shape)

(49800, 784) (10200, 784) (10000, 784)
(49800,) (10200,) (10000,)


### Cambiar etiquetas a codificación 'One Hot'

In [ ]:
from tensorflow.keras.utils import to_categorical

print(y_train[1])
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)
print(y_train[1])

0
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Crear Modelo

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

# Especificar número de entradas
inputs = Input(shape=(784,))

# Crear capas de neuronas
x = Dense(784, activation='sigmoid')(inputs)
x = Dense(350, activation='sigmoid')(x)
x = Dense(196, activation='sigmoid')(x)
x = Dense(70, activation='sigmoid')(x)
x = Dense(10, activation='softmax')(x) #Se usa softmax para clasificar correctamente muchas clases

model = Model(inputs=inputs, outputs=x)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 350)               274750    
                                                                 
 dense_2 (Dense)             (None, 196)               68796     
                                                                 
 dense_3 (Dense)             (None, 70)                13790     
                                                                 
 dense_4 (Dense)             (None, 10)                710       
                                                                 
Total params: 973,486
Trainable params: 973,486
Non-trainable

In [ ]:
# Configurar el modelo
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-3), 
              metrics=['accuracy'])

### Entrenar el modelo

In [ ]:
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=50, batch_size = 32, verbose=1)

Epoch 1/50
1557/1557 [==============================] - 20s 12ms/step - loss: 0.4952 - accuracy: 0.8521 - val_loss: 0.2087 - val_accuracy: 0.9403
Epoch 2/50
1557/1557 [==============================] - 18s 12ms/step - loss: 0.1599 - accuracy: 0.9535 - val_loss: 0.1414 - val_accuracy: 0.9609
Epoch 3/50
1557/1557 [==============================] - 20s 13ms/step - loss: 0.1055 - accuracy: 0.9689 - val_loss: 0.1212 - val_accuracy: 0.9656
Epoch 4/50
1557/1557 [==============================] - 20s 13ms/step - loss: 0.0766 - accuracy: 0.9773 - val_loss: 0.1007 - val_accuracy: 0.9707
Epoch 5/50
1557/1557 [==============================] - 19s 12ms/step - loss: 0.0603 - accuracy: 0.9816 - val_loss: 0.0923 - val_accuracy: 0.9742
Epoch 6/50
1557/1557 [==============================] - 20s 13ms/step - loss: 0.0449 - accuracy: 0.9864 - val_loss: 0.0934 - val_accuracy: 0.9735
Epoch 7/50
1557/1557 [==============================] - 23s 15ms/step - loss: 0.0360 - accuracy: 0.9886 - val_loss: 0.1037 -

### Predicciones

In [ ]:
# Predecir con el modelo
pred = model.predict(x_test)
# Devolver la posición del valor más alto de cada predicción (ya que está en One Hot)
pred = np.argmax(pred, axis = 1) 
print(pred)
print(pred.shape)

[7 2 1 ... 4 5 6]
(10000,)


In [ ]:
# Comparar y_test con las predicciones
score = model.evaluate(x_test, y_test)
print('Loss:', score[0]) 
print('Accuracy:', score[1])

313/313 [==============================] - 1s 5ms/step - loss: 0.1194 - accuracy: 0.9811
Loss: 0.11939635872840881
Accuracy: 0.9811000227928162
